In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
from qutip import *
from estimation import *
from data_processing import *
from scipy.io import savemat
from targets import *
import exp_params

USING Q-FUNCTION POINTS


### Check yourself: print function, dimension, and number of displacements

In [2]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

Q
D=6
nD=35


### State list

In [3]:
state_list = exp_params.fock_state_list
print(state_list)

['fock0' 'fock01' 'fock0i1' 'fock02' 'fock0i2' 'fock1' 'fock03' 'fock0i3'
 'fock12' 'fock1i2' 'fock04' 'fock0i4' 'fock13' 'fock1i3' 'fock2' 'fock05'
 'fock0i5' 'fock14' 'fock1i4' 'fock23' 'fock2i3' 'fock15' 'fock1i5'
 'fock24' 'fock2i4' 'fock3' 'fock25' 'fock2i5' 'fock34' 'fock3i4' 'fock35'
 'fock3i5' 'fock4' 'fock45' 'fock4i5' 'fock5']


### Experimental data directory

In [4]:
if exp_params.nD == exp_params.D**2 - 1:
    exp_data_directory = r"data\fidelity_v_dimension\Q\minimal\D" + str(exp_params.D)
elif exp_params.nD == 2 * (exp_params.D**2 - 1):
    exp_data_directory = r"data\fidelity_v_dimension\Q\double\D" + str(exp_params.D)
all_files = np.array(os.listdir(exp_data_directory))

### Directory for target states generated from grape (we don't use ideal when we're processing experimental data)

In [5]:
# GRAPE generated target states
target_states_directory = r"target_states"

### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [6]:
# Load inverse map variables for converting pe to rho
map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_Q.npz"
)
W = map_variables["W"]
beta = map_variables["beta"]


### State estimation with MLE and Bayes

In [7]:
F_MLE = np.zeros(len(state_list), dtype=float)
F_Bayes_BME = np.zeros(len(state_list), dtype=float)

final_rho_BME = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)

final_rho_MLE = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)


In [8]:
for j, state_name in enumerate(state_list):  # State List
    data = np.zeros([exp_params.nD])  # Initialize data vector

    for point in range(1, exp_params.nD + 1):  # Point1 to Point35
        # NON-T2 DATA
        ending = (
            "D="
            + str(exp_params.D)
            + "_grape_"
            + str(state_name)
            + "_point"
            + str(point)
            + ".h5"
        )
      
        matching = [
            filename for filename in all_files if filename.endswith(str(ending))
        ]

        filepath = exp_data_directory + "/" + matching[0]
        file = h5py.File(filepath, "r")

        signal = post_selection(filepath)

        if state_name in exp_params.pes_after_grape[:, 0]:
            pge = exp_params.pes_after_grape[
                (exp_params.pes_after_grape[:, 0] == state_name), 1
            ].astype(float)

        else:
            pge = exp_params.pe_after_grape_avg

        aux = (signal - pge) / (1 - 2 * pge)
        data[point - 1] = aux

    rho_est, qRho_est = get_LS_and_MLE_rho_est(
        data, W, beta, exp_params.D, exp_params.nD
    )

    rho_tar_qc = Y_target(
        state_name, target_states_directory, qdim=exp_params.qdim, cdim=exp_params.cdim
    )

    rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])
    rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()

    Fmean_bayes, Fstd_bayes, rho_BME = bayesian_rho_est(
        numSamp=2**10,
        N=exp_params.N_exp,
        rho_tar=rho_tar_c,
        rhoLS=rho_est.full(),
    )

    F_Bayes_BME[j] = fidelity(rho_tar_c, Qobj(rho_BME)) ** 2
    F_MLE[j] = fidelity(rho_tar_c, qRho_est) ** 2

    final_rho_BME[j, :, :] = rho_BME
    final_rho_MLE[j, :, :] = qRho_est

    print(f"state {j}: {state_name}: {F_Bayes_BME[j]}")

C:\Users\clara\AppData\Local\Temp\ipykernel_20872\2612542100.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data[point - 1] = aux


state 0: fock0: 0.9636455353096797
state 1: fock01: 0.9503816194459913


### Save results in an npz

In [ ]:
if exp_params.nD == exp_params.D**2 - 1:
    np.savez(
        f"results_dimensions\Q\minimal\Bayes_F_D={exp_params.D}_nD={exp_params.nD}_BME_MLE_rho_Q.npz",
        F_Bayes_BME=F_Bayes_BME,
        F_MLE=F_MLE,
        final_rho_BME=final_rho_BME,
        final_rho_MLE=final_rho_MLE,
    )
elif exp_params.nD == 2 * (exp_params.D**2 - 1):
    np.savez(
        f"results_dimensions\Q\double\Bayes_F_D={exp_params.D}_nD={exp_params.nD}_BME_MLE_rho_Q.npz",
        F_Bayes_BME=F_Bayes_BME,
        F_MLE=F_MLE,
        final_rho_BME=final_rho_BME,
        final_rho_MLE=final_rho_MLE,
    )